In [12]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
base_path = '/content/gdrive/MyDrive/upwork/applied_cv/'

In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline


In [4]:
!pwd
os.chdir(base_path)
!pwd

/content
/content/gdrive/MyDrive/upwork/applied_cv


In [ ]:
def match_keypoints(image_original, image_compare):
  original = cv2.imread(image_original)
  image_to_compare = cv2.imread(image_compare)

  # Check if two images are equals
  if original.shape == image_to_compare.shape:
    print("The images have same size and channels")
    difference = cv2.subtract(original, image_to_compare)
    b, g, r = cv2.split(difference)

    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
      print("The images are completely equal")
    else:
      print("The images are not equal")

  # Check for similarities between the two images
  sift = cv2.xfeatures2d.SIFT_create()
  kp_1, desc_1 = sift.detectAndCompute(original, None)
  kp_2, desc_2 = sift.detectAndCompute(image_to_compare, None)

  index_params = dict(algorithm=0, trees=5)
  search_params = dict()
  flann = cv2.FlannBasedMatcher(index_params, search_params)

  matches = flann.knnMatch(desc_1, desc_2, k=2)

  good_points = []
  ratio = 0.6

  for m, n in matches:
	if m.distance < ratio*n.distance:
		good_points.append(m)
  
  print(f"Total number of keypoints: {len(good_points)}")
  print(f"Total number of matches: {len(matches)}")
  print(f"Ratio of matches to keypoints: {len(matches)/len(good_points)}")
  
  result = cv2.drawMatches(original, kp_1, image_to_compare, kp_2, good_points, None)

  cv2.imshow("result", result)
  cv2.imshow("Original", original)
  cv2.imshow("Duplicate", image_to_compare)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

  return result